# Code to Read CSV and Excel Fiel

## Begin Load Packages

## End Load Packages

## Begin Read Excel from
1. from sheet in excel
2. read into dataframe
3. Start with row that has first value of keyword in the column
4. Save to csv

In [1]:
def process_excel_from_keyword_robust(
    input_file='data.xlsx',
    source_dir='.',
    output_file='output',
    output_dir='.',
    column_name='Column1',
    keyword='START',
    sheet_name=0
):
    """Process Excel with better error handling"""
    
    try:
        # Build and check input path
        input_path = os.path.join(source_dir, input_file)
        if not os.path.exists(input_path):
            print(f"❌ File not found: {input_path}")
            return None
        
        # Read Excel
        df = pd.read_excel(input_path, sheet_name=sheet_name)
        
        # Check if column exists
        if column_name not in df.columns:
            print(f"❌ Column '{column_name}' not found. Available columns: {list(df.columns)}")
            return None
        
        # Find keyword
        mask = df[column_name] == keyword
        if not mask.any():
            print(f"❌ Keyword '{keyword}' not found in column '{column_name}'")
            return pd.DataFrame()
        
        # Filter from first occurrence
        first_index = df[mask].index[0]
        filtered_df = df.iloc[first_index:]
        
        # Save with date
        date_suffix = datetime.now().strftime('%Y%m%d')
        output_path = os.path.join(output_dir, f"{output_file}_{date_suffix}.csv")
        filtered_df.to_csv(output_path, index=False)
        
        print(f"✓ Success! Saved {len(filtered_df)} rows to: {output_path}")
        return filtered_df
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

## End read excel and convert to csv

## Begin Combine Excel Files
1. read a set of files
2. all files have date as first column
3. output to one csv with all date
4. output to one csve with common dates only

In [2]:
import pandas as pd
from datetime import datetime
from functools import reduce

def combine_excel_files(file_list, output_prefix="combined"):
    # Get current date for output file naming
    date_stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Read all Excel files into a list of dataframes
    dfs = []
    for i, file in enumerate(file_list):
        df = pd.read_excel(file)
        # Add suffix to columns (except Date) to avoid naming conflicts
        df.columns = ['Date'] + [f"{col}_file{i+1}" for col in df.columns[1:]]
        dfs.append(df)
    
    # File 1: Superset - Full outer join on Date
    df_superset = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dfs)
    df_superset.sort_values('Date', inplace=True)
    
    # File 2: Common dates - Inner join on Date
    df_common = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'), dfs)
    df_common.sort_values('Date', inplace=True)
    
    # Save to CSV files
    df_superset.to_csv(f"{output_prefix}_all_dates_{date_stamp}.csv", index=False)
    df_common.to_csv(f"{output_prefix}_common_dates_{date_stamp}.csv", index=False)
    
    print(f"Created: {output_prefix}_all_dates_{date_stamp}.csv (rows: {len(df_superset)})")
    print(f"Created: {output_prefix}_common_dates_{date_stamp}.csv (rows: {len(df_common)})")

# # Example usage
# excel_files = ['file1.xlsx', 'file2.xlsx', 'file3.xlsx']
# combine_excel_files(excel_files, "merged_data")

In [ ]:
# Example usage
from pathlib import Path
# Relative to notebook
# "C:\Users\Hassan\Dropbox\GithubRepo\MarketPred\DownloadedCleanedInput\Breadth_MA200.xlsx"
data_folder = Path("../MarketPred/DownloadedCleanedInput")
file1 = data_folder / "Breadth_MA200.xlsx"
file2 = data_folder / "Cape.xlsx"
file3 = data_folder / "PriceVolume.xlsx"
file4 = data_folder / "Spread.xlsx"
file5 = data_folder / "Yield_Curve.xlsx"

excel_files = [file1, file2, file3, file4, file5]
combine_excel_files(excel_files, "merged_data")

### Begin normalize date format
1. change code below to fill na

In [8]:
import pandas as pd
from functools import reduce
from datetime import datetime
import os

def merge_excel_files_normalized_dates(
    file_paths,
    date_column,
    merge_type='outer',
    output_dir='.',
    output_prefix='merged'
):
    """
    Merge multiple Excel files on a date column with date format normalization.
    
    Parameters:
    -----------
    file_paths : list
        List of Excel file paths to merge
    date_column : str
        Name of the date column to merge on
    merge_type : str
        'outer' for superset (all dates), 'inner' for intersection (common dates only)
    output_dir : str
        Directory to save output file (default: current directory)
    output_prefix : str
        Prefix for output filename
    
    Returns:
    --------
    pd.DataFrame : Merged dataframe
    """
    
    def normalize_date_column(df, date_col):
        """Convert date column to standard datetime format"""
        try:
            # Try to convert to datetime, handling multiple formats
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            # Optionally normalize to just date (remove time component)
            df[date_col] = df[date_col].dt.date
        except Exception as e:
            print(f"Warning: Could not normalize dates - {e}")
        return df
    
    # Validate files exist
    for fp in file_paths:
        if not os.path.exists(fp):
            raise FileNotFoundError(f"File not found: {fp}")
    
    # Read all files
    dfs = []
    for fp in file_paths:
        try:
            df = pd.read_excel(fp)
            
            # Validate date column exists
            if date_column not in df.columns:
                raise ValueError(f"Column '{date_column}' not found in {fp}")
            
            # Normalize date column
            df = normalize_date_column(df, date_column)
            
            # Add source file info to column names (except date column)
            file_name = os.path.splitext(os.path.basename(fp))[0]
            df.columns = [date_column if col == date_column else f"{col}_{file_name}" 
                         for col in df.columns]
            
            dfs.append(df)
            print(f"✓ Loaded and normalized: {fp}")
            
        except Exception as e:
            print(f"✗ Error reading {fp}: {e}")
            raise
    
    # Merge all dataframes
    merged_df = reduce(
        lambda left, right: pd.merge(left, right, on=date_column, how=merge_type),
        dfs
    )
    
    # Sort by date
    merged_df = merged_df.sort_values(date_column).reset_index(drop=True)
    
    # Generate output filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(
        output_dir, 
        f"{output_prefix}_{merge_type}_{timestamp}.csv"
    )
    
    # Save to CSV
    merged_df.to_csv(output_file, index=False)
    print(f"\n✓ Merged file saved: {output_file}")
    print(f"  Total rows: {len(merged_df)}")
    print(f"  Total columns: {len(merged_df.columns)}")
    
    return merged_df

# # Example usage:
# file_paths = [
#     'file1.xlsx',
#     'file2.xlsx',
#     'file3.xlsx'
# ]

# # For superset (all dates from all files)
# df_superset = merge_excel_files_normalized_dates(
#     file_paths=file_paths,
#     date_column='Date',
#     merge_type='outer',
#     output_prefix='superset'
# )

# # For intersection (only common dates)
# df_intersection = merge_excel_files_normalized_dates(
#     file_paths=file_paths,
#     date_column='Date',
#     merge_type='inner',
#     output_prefix='intersection'
# )

In [9]:
# Example usage:
data_folder = Path("../MarketPred/DownloadedCleanedInput")
file1 = data_folder / "Breadth_MA200.xlsx"
file2 = data_folder / "Cape.xlsx"
file3 = data_folder / "PriceVolume.xlsx"
file4 = data_folder / "Spread.xlsx"
file5 = data_folder / "Yield_Curve.xlsx"

file_paths = [file1, file2, file3, file4, file5]
# file_paths = [
#     'file1.xlsx',
#     'file2.xlsx',
#     'file3.xlsx'
# ]

# For superset (all dates from all files)
df_superset = merge_excel_files_normalized_dates(
    file_paths=file_paths,
    date_column='Date',
    merge_type='outer',
    output_prefix='superset'
)

# For intersection (only common dates)
df_intersection = merge_excel_files_normalized_dates(
    file_paths=file_paths,
    date_column='Date',
    merge_type='inner',
    output_prefix='intersection'
)

✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Breadth_MA200.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Cape.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\PriceVolume.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Spread.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Yield_Curve.xlsx

✓ Merged file saved: .\superset_outer_20251221_090106.csv
  Total rows: 52695
  Total columns: 7
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Breadth_MA200.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Cape.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\PriceVolume.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Spread.xlsx
✓ Loaded and normalized: ..\MarketPred\DownloadedCleanedInput\Yield_Curve.xlsx

✓ Merged file saved: .\intersection_inner_20251221_090110.csv
  Total rows: 6015
  Total columns: 7


### End normalize date format

## End Combine Excel files